In [ ]:
from pathlib import Path

import matplotlib.pyplot as plt
import numpy as np
import xarray as xr

from sdm_eurec4a.visulization import set_custom_rcParams, adjust_lightness_array, ncols_nrows_from_N


from sdm_eurec4a import RepositoryPath

In [ ]:
def adjust_spines(ax, visible_spines, position=("outward", 5)):
    ax.label_outer(remove_inner_ticks=False)

    for loc, spine in ax.spines.items():
        if loc in visible_spines:
            spine.set_position(position)  # outward by 10 points
        else:
            spine.set_visible(False)

In [ ]:
plt.style.use("default")
default_colors = set_custom_rcParams()
darker_colors = adjust_lightness_array(default_colors, 0.75)

REPOSITORY_ROOT = RepositoryPath("levante").get_repo_dir()

fig_path = REPOSITORY_ROOT / Path("results/compare_cluster_and_clouds/shallow_only")
fig_path.mkdir(parents=True, exist_ok=True)

In [ ]:
mask_name = "rain_mask"
min_holes = 5
clusters = xr.open_dataset(
    REPOSITORY_ROOT
    / Path(
        f"data/observation/cloud_composite/processed/identified_clouds/identified_clusters_{mask_name}_{min_holes}.nc"
    )
)
clouds = xr.open_dataset(
    REPOSITORY_ROOT
    / Path(
        f"data/observation/cloud_composite/processed/identified_clouds/identified_clouds_{mask_name}.nc"
    )
)

# select only clouds which are between 800 and 1100 m
clouds = clouds.where((clouds.alt <= 1500), drop=True)
clusters = clusters.where((clusters.alt <= 1500), drop=True)

distance_clouds = xr.open_dataset(
    REPOSITORY_ROOT
    / Path(f"data/observation/combined/distance/distance_dropsondes_identified_clouds_rain_mask.nc")
)

distance_clusters = xr.open_dataset(
    REPOSITORY_ROOT
    / Path(f"data/observation/combined/distance/distance_dropsondes_identified_clusters_rain_mask_5.nc")
)

cloud_composite = xr.open_dataset(
    REPOSITORY_ROOT / Path("data/observation/cloud_composite/processed/cloud_composite_si_units.nc"),
    chunks={"time": 1000},
)


drop_sondes = xr.open_dataset(
    REPOSITORY_ROOT / Path("data/observation/dropsonde/processed/drop_sondes.nc")
)

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(9, 4.5))
# bins = np.arange(0,300,50)
ax.hist(clouds["alt"], bins=100, orientation="horizontal", alpha=0.7)  # , bins)
ax.hist(clusters["alt"], bins=100, orientation="horizontal", alpha=0.7)  # bins)

ax.set_ylabel("Altitude [m]")
ax.set_xlabel("Frequency")

fig.suptitle(f"Altitude distribution of identified clouds and clusters based on {mask_name}")

fig.savefig(fig_path / Path(f"altitude_distribution_{mask_name}.png"), dpi=300)

Create bar plot of 
- duration 
- horizontal extent
- vertical extent

In [ ]:
fig, axs = plt.subplots(nrows=3, ncols=2, figsize=np.array([8, 4.5]) * 1.5)

axs_top = axs[0]
axs_mid = axs[1]
axs_bot = axs[2]

# At the

bins = np.arange(0, 15, 1)
axs_top[0].hist(
    clouds["duration"].dt.seconds, bins=bins, alpha=0.5, color=default_colors[0], label="Clouds"
)
axs_top[0].hist(
    clusters["duration"].dt.seconds, bins=bins, alpha=0.5, color=default_colors[1], label="Clusters"
)

bins = np.arange(0, 351, 5)
axs_top[1].hist(
    clouds["duration"].dt.seconds, bins=bins, alpha=0.5, color=default_colors[0], label="Clouds"
)
axs_top[1].hist(
    clusters["duration"].dt.seconds, bins=bins, alpha=0.5, color=default_colors[1], label="Clusters"
)
axs_top[1].set_yscale("log")

for ax in axs_top:
    # adjust_spines(ax, ["left", "bottom"])
    ax.set_xlabel("ATR flight duration [s]")
    ax.set_ylabel("Frequency")

bins = np.arange(0, 1501, 50)
axs_mid[0].hist(
    clouds["horizontal_extent"], bins=bins, alpha=0.5, color=default_colors[0], label="Clouds"
)
axs_mid[0].hist(
    clusters["horizontal_extent"], bins=bins, alpha=0.5, color=default_colors[1], label="Clusters"
)

bins = np.arange(0, 35001, 1000)
axs_mid[1].hist(
    clouds["horizontal_extent"], bins=bins, alpha=0.5, color=default_colors[0], label="Clouds"
)
axs_mid[1].hist(
    clusters["horizontal_extent"], bins=bins, alpha=0.5, color=default_colors[1], label="Clusters"
)
axs_mid[1].set_yscale("log")

for ax in axs_mid:
    # adjust_spines(ax, ["left", "bottom"])
    ax.set_xlabel("Horizontal extent [m]")
    ax.set_ylabel("Frequency")

bins = np.arange(0, 11, 1)
axs_bot[0].hist(clouds["vertical_extent"], bins=bins, alpha=0.5, color=default_colors[0], label="Clouds")
axs_bot[0].hist(
    clusters["vertical_extent"], bins=bins, alpha=0.5, color=default_colors[1], label="Clusters"
)

bins = np.arange(0, 1001, 20)
axs_bot[1].hist(clouds["vertical_extent"], bins=bins, alpha=0.5, color=default_colors[0], label="Clouds")
axs_bot[1].hist(
    clusters["vertical_extent"], bins=bins, alpha=0.5, color=default_colors[1], label="Clusters"
)
axs_bot[1].set_yscale("log")

for ax in axs_bot:
    # adjust_spines(ax, ["left", "bottom"])
    ax.set_xlabel("Vertical extent [m]")
    ax.set_ylabel("Frequency")

for ax in axs.flatten():
    # adjust_spines(ax, ["left", "bottom"])
    ax.legend()

fig.suptitle(
    f"Horizontal extent of clouds. All clouds and clusters. Based on '{mask_name}'\nFor clusters: Holes of {min_holes} seconds are ignored."
)
fig.tight_layout()
fig.savefig(fig_path / Path(f"extent_distribution_{mask_name}.png"), dpi=300)

In [ ]:
research_days = np.unique(clouds["time"].dt.dayofyear)

# do the plotting
fig, axs = plt.subplots(
    sharex=True, sharey=True, **ncols_nrows_from_N(len(research_days)), figsize=(16, 9)
)
axs_flat = axs.flatten()


bins = np.arange(0, 30000, 1000)
for ax, day in zip(axs_flat, research_days):
    t = clouds["time"].where(clouds["time"].dt.dayofyear == day, drop=True)
    ds_clouds = clouds.sel(time=t)
    ax.hist(
        ds_clouds["horizontal_extent"], bins=bins, alpha=0.5, color=default_colors[0], label="Clouds"
    )

    t = clusters["time"].where(clusters["time"].dt.dayofyear == day, drop=True)
    ds_clusters = clusters.sel(time=t)
    ax.hist(
        ds_clusters["horizontal_extent"], bins=bins, alpha=0.5, color=default_colors[1], label="Clusters"
    )

    ax.set_title(ds_clouds.time[0].dt.strftime("%Y-%m-%d").data)
    ax.legend()
    ax.set_yscale("log")

for ax in axs[-1, :]:
    ax.set_xlabel("Horizontal extent [m]")
for ax in axs[:, 0]:
    ax.set_ylabel("Frequency")

# tidy up the figure
fig.suptitle(
    f"Horizontal extent of clouds. Clouds and clusters per day. Based on '{mask_name}'\nFor clusters: Holes of {min_holes} seconds are ignored."
)

fig.tight_layout()

fig.savefig(fig_path / f"horizontal_extent_{mask_name}_per_day_full_range.png", dpi=300)

In [ ]:
research_days = np.unique(clouds["time"].dt.dayofyear)

# do the plotting
fig, axs = plt.subplots(
    sharex=True, sharey=True, **ncols_nrows_from_N(len(research_days)), figsize=(16, 9)
)
axs_flat = axs.flatten()


bins = np.arange(0, 1000, 50)
for ax, day in zip(axs_flat, research_days):
    t = clouds["time"].where(clouds["time"].dt.dayofyear == day, drop=True)
    ds_clouds = clouds.sel(time=t)
    ax.hist(
        ds_clouds["horizontal_extent"], bins=bins, alpha=0.5, color=default_colors[0], label="Clouds"
    )

    t = clusters["time"].where(clusters["time"].dt.dayofyear == day, drop=True)
    ds_clusters = clusters.sel(time=t)
    ax.hist(
        ds_clusters["horizontal_extent"], bins=bins, alpha=0.5, color=default_colors[1], label="Clusters"
    )

    ax.set_title(ds_clouds.time[0].dt.strftime("%Y-%m-%d").data)
    ax.legend()

for ax in axs[-1, :]:
    ax.set_xlabel("Horizontal extent [m]")
for ax in axs[:, 0]:
    ax.set_ylabel("Frequency")

# tidy up the figure
fig.suptitle(
    f"Horizontal extent of clouds of less then 1000 m. Clouds and clusters per day. Based on '{mask_name}'\nFor clusters: Holes of {min_holes} seconds are ignored."
)

fig.tight_layout()

fig.savefig(fig_path / f"horizontal_extent_{mask_name}_per_day_less_1000m.png", dpi=300)